In [1]:
# Import modules
import pandas as pd
import json
import requests

import matplotlib.pyplot as plt
import numpy as np

# Google API Key
from config import gkey

In [2]:
# Open csv file for reading City Survey Master Data 1996-2017 csv file downloaded from http://sfgov.org/citysurvey/file/95.
# Open csv file from https://www.zillow.com/research/data/
# Save path to data set in a variable
data_file = "Resources/City Survey MASTER Data 2015-2017 Copy.csv"
data_file2 = "Resources/Zip_Zhvi_SingleFamilyResidence.csv"

In [3]:
# Use Pandas to read sfgov.org data and zillow medium single family home price
city_survey_pd = pd.read_csv(data_file, dtype={'zipcode':'str', 'deduc' : 'int'})
city_survey_pd.head()

home_pd = pd.read_csv(data_file2)
home_pd.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02
0,84654,60657,Chicago,IL,Chicago,Cook,1,420800.0,423500.0,426200.0,...,1072300,1062400,1051500,1047200,1047300,1047300,1049600,1050900,1049400,1049600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,NaN,NaN,NaN,...,317500,318400,318500,319400,322400,324800,325400,325000,325100,325000
2,84616,60614,Chicago,IL,Chicago,Cook,3,542400.0,546700.0,551700.0,...,1522500,1514200,1511300,1513400,1513300,1508000,1503700,1495500,1487500,1487000
3,93144,79936,El Paso,TX,El Paso,El Paso,4,70900.0,71200.0,71100.0,...,114000,114100,113800,113700,114200,114300,114000,114000,114300,114200
4,91733,77084,Houston,TX,Houston,Harris,5,76700.0,76500.0,76000.0,...,158800,160700,161400,162600,164100,164400,164200,163800,164000,164400


In [4]:
# Select specific columns
sf_2017_pd = city_survey_pd[city_survey_pd.year == 2017]
sf_pd = sf_2017_pd[['year', 'zipcode', 'dage', 'dethnic', 'dlivedsf', 'deduc', 'dincome', 'movesf', 'ownrenhm']].copy()
sf_pd['zipcode'] = sf_pd['zipcode'].astype(int) 
sf_pd.head()

sf_pd.dtypes

year        int64
zipcode     int32
dage        int64
dethnic     int64
dlivedsf    int64
deduc       int32
dincome     int64
movesf      int64
ownrenhm    int64
dtype: object

In [5]:
# Count rows to see if there's any missing values.
sf_pd.count()

year        2166
zipcode     2166
dage        2166
dethnic     2166
dlivedsf    2166
deduc       2166
dincome     2166
movesf      2166
ownrenhm    2166
dtype: int64

In [6]:
# Select San Francisco single family home prices from Zillow file
# Get aveage home price for 2017

sfhome_pd = home_pd[home_pd.City == 'San Francisco']

AvePrice = sfhome_pd[['2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12']].mean(axis=1)
sfhome_pd['AvePrice'] = AvePrice

homeprice_pd = sfhome_pd[['RegionName', 'City', 'State', 'AvePrice']].copy()

homeprice_pd = homeprice_pd.rename(columns={'RegionName': 'zipcode'})

homeprice_pd.head()

C:\Users\DCPC\Anaconda3\envs\PythonData2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,zipcode,City,State,AvePrice
8,94109,San Francisco,CA,3.942708e+06
63,94110,San Francisco,CA,1.428692e+06
313,94122,San Francisco,CA,1.301808e+06
477,94112,San Francisco,CA,9.375333e+05
715,94115,San Francisco,CA,4.077917e+06


In [7]:
homeprice_pd.dtypes

zipcode       int64
City         object
State        object
AvePrice    float64
dtype: object

In [8]:
# Add columns for lat and lng
homeprice_pd["Lat"] = ""
homeprice_pd["Lng"] = ""
homeprice_pd.head()

,zipcode,City,State,AvePrice,Lat,Lng
8,94109,San Francisco,CA,3.942708e+06,,
63,94110,San Francisco,CA,1.428692e+06,,
313,94122,San Francisco,CA,1.301808e+06,,
477,94112,San Francisco,CA,9.375333e+05,,
715,94115,San Francisco,CA,4.077917e+06,,


In [9]:
# create a params dict that will be updated with new zip code iteration
params = {"key": gkey}

# Loop through the homeprice_pd and run a lat/long search for each zip code
for index, row in homeprice_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    zipcode = row['zipcode']

    # update address key value
    params['address'] = f"{zipcode}"

    # make request, print url
    cities_lat_lng = requests.get(base_url, params=params)

    # convert to json
    cities_lat_lng = cities_lat_lng.json()

    homeprice_pd.set_value(
        index, "Lat", cities_lat_lng["results"][0]["geometry"]["location"]["lat"])
    homeprice_pd.set_value(
        index, "Lng", cities_lat_lng["results"][0]["geometry"]["location"]["lng"])

# Visualize to confirm lat lng appear
homeprice_pd.head()

C:\Users\DCPC\Anaconda3\envs\PythonData2\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\DCPC\Anaconda3\envs\PythonData2\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,zipcode,City,State,AvePrice,Lat,Lng
8,94109,San Francisco,CA,3.942708e+06,37.793,-122.421
63,94110,San Francisco,CA,1.428692e+06,37.7486,-122.418
313,94122,San Francisco,CA,1.301808e+06,37.7597,-122.475
477,94112,San Francisco,CA,9.375333e+05,37.7225,-122.441
715,94115,San Francisco,CA,4.077917e+06,37.7878,-122.438


In [10]:
# Include single family home average price and latitude and longitude.
combined_pd = pd.merge(sf_pd, homeprice_pd, on='zipcode')

combined_pd.head()

,year,zipcode,dage,dethnic,dlivedsf,deduc,dincome,movesf,ownrenhm,City,State,AvePrice,Lat,Lng
0,2017,94109,7,4,6,3,2,4,2,San Francisco,CA,3.942708e+06,37.793,-122.421
1,2017,94109,2,4,3,4,6,2,2,San Francisco,CA,3.942708e+06,37.793,-122.421
2,2017,94109,3,4,2,4,6,2,2,San Francisco,CA,3.942708e+06,37.793,-122.421
3,2017,94109,2,2,2,4,3,2,2,San Francisco,CA,3.942708e+06,37.793,-122.421
4,2017,94109,5,4,1,4,8,1,3,San Francisco,CA,3.942708e+06,37.793,-122.421


In [11]:
# Select other cities to make single family home price comparison.
list_of_cities = ["San Francisco", "Los Angeles", "San Diego", "Sacramento", "Portland", "Seattle"]

home_pd= home_pd[home_pd['City'].isin(list_of_cities)]

home_pd

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02
8,97564,94109,San Francisco,CA,San Francisco,San Francisco,9,NaN,NaN,NaN,...,3919100,3956400,3981600,3953600,3893200,3892600,3957300,3969900,3902700,3861400
23,96027,90046,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,24,374500.0,374400.0,374700.0,...,1650300,1663100,1665600,1668800,1676100,1720600,1765900,1774000,1775800,1805400
62,96015,90034,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,63,234500.0,234500.0,235400.0,...,1156000,1156800,1151500,1155500,1171700,1193500,1224800,1251900,1274900,1294400
63,97565,94110,San Francisco,CA,San Francisco,San Francisco,64,NaN,NaN,NaN,...,1413800,1424700,1437100,1436900,1432900,1453100,1494000,1512400,1509400,1511200
72,96025,90044,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,73,115600.0,114700.0,114400.0,...,400100,404900,408200,410900,414900,418700,422000,424100,426700,429800
92,96007,90026,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,93,145400.0,144700.0,144900.0,...,882600,891400,892400,894800,904200,918500,932700,943100,953200,962400
118,96047,90066,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,119,243000.0,243800.0,243900.0,...,1291800,1296800,1292800,1296100,1309100,1331600,1347000,1353700,1359000,1369900
130,98432,95823,Sacramento,CA,Sacramento,Sacramento,131,88700.0,88200.0,87700.0,...,257000,260100,262600,264000,265200,267300,268900,269800,272000,274700
146,99563,98103,Seattle,WA,Seattle,King,147,186500.0,185500.0,184500.0,...,803300,813800,814800,816300,826000,840400,854000,861900,868600,873500
148,96667,92126,San Diego,CA,San Diego,San Diego,149,156900.0,156300.0,155500.0,...,568900,576600,583000,587300,588200,590200,593100,596600,603300,609600


In [12]:
home_price = home_pd.groupby(["City"]).mean()
home_price

,RegionID,RegionName,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,...,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02
City,,,,,,,,,,,,,,,,,,,,,
Los Angeles,96170.119565,90589.510870,2414.032609,230146.913580,230508.641975,230354.320988,230061.728395,229946.913580,229818.518519,229787.654321,...,9.504630e+05,9.547793e+05,9.566728e+05,9.618913e+05,9.712033e+05,9.858120e+05,1.001565e+06,1.011500e+06,1.017961e+06,1.025458e+06
Portland,91199.058824,78387.029412,4995.647059,135258.823529,136188.235294,137279.411765,138273.529412,139191.176471,140082.352941,141135.294118,...,4.475588e+05,4.478471e+05,4.480824e+05,4.487735e+05,4.494265e+05,4.485529e+05,4.483000e+05,4.500588e+05,4.527706e+05,4.546882e+05
Sacramento,112776.428571,95817.809524,4054.523810,107415.000000,107385.000000,107255.000000,107125.000000,106840.000000,106540.000000,106365.000000,...,3.283095e+05,3.320286e+05,3.339000e+05,3.346048e+05,3.355714e+05,3.385810e+05,3.420333e+05,3.444714e+05,3.476667e+05,3.513524e+05
San Diego,96523.580645,91675.000000,2538.903226,207262.068966,206793.103448,206227.586207,205679.310345,205189.655172,205320.689655,205989.655172,...,7.247581e+05,7.347194e+05,7.415645e+05,7.452677e+05,7.475806e+05,7.541581e+05,7.634129e+05,7.708258e+05,7.789290e+05,7.872968e+05
San Francisco,97573.631579,94119.368421,1780.789474,265180.000000,266630.000000,268140.000000,269750.000000,271280.000000,272220.000000,272970.000000,...,1.910626e+06,1.918858e+06,1.935658e+06,1.945379e+06,1.940700e+06,1.960511e+06,2.017942e+06,2.046621e+06,2.038184e+06,2.037684e+06
Seattle,99577.631579,98120.473684,2429.578947,195900.000000,194978.947368,194252.631579,193968.421053,194078.947368,194531.578947,195305.263158,...,8.120421e+05,8.266000e+05,8.336789e+05,8.368105e+05,8.423947e+05,8.522579e+05,8.639632e+05,8.759842e+05,8.907211e+05,9.005895e+05


In [13]:
# Save both sets of data to csv files.
combined_pd.to_csv('sfdata.csv', index=False, header=True)
home_price.to_csv('homeprice.csv', index=True, header=True)